In [ ]:
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


Read in

In [ ]:
img_path = "./data/images/dset/gt/img_0006_gt.jpg"
image_bgr = cv2.imread(img_path)
img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)


Segment and Spill

In [ ]:
seg_model = "./preprocessing/models/selfie_segmenter.tflite"
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)

base_options = python.BaseOptions(model_asset_path=seg_model)
options = vision.ImageSegmenterOptions(base_options=base_options,
                                       output_category_mask=False,
                                       output_confidence_masks=True)

with vision.ImageSegmenter.create_from_options(options=options) as segmenter:
    result = segmenter.segment(image=mp_image)
    mask = result.confidence_masks[0].numpy_view()  # Shape: [H, W], values 0.0 - 1.0



In [ ]:
print(type(mask))

In [ ]:
spill_strength = 0.4
saturation_boost = 18
threshold = 0.3
blur_kernel = (21, 21)
# Need Confidence mask (mask) and bgr image (image_bgr)


In [ ]:
# Binarise the confidence mask
foreground_mask = (mask > threshold).astype(np.uint8)

# Create edge mask
edge_mask = cv2.distanceTransform(1 - foreground_mask.astype(np.uint8), cv2.DIST_L2, 3)
edge_mask = cv2.normalize(edge_mask, None, 0, 1.0, cv2.NORM_MINMAX)

# Invert distance so that edges = 1, centre = 0
spill_mask = 1.0 - edge_mask
spill_mask = np.clip(spill_mask, 0, 1)

# Blur to create soft transition
spill_mask = cv2.GaussianBlur(spill_mask, blur_kernel, 0)

# Steepen falloff to reduce inner-body spill
spill_mask = np.power(spill_mask, 2.5)

# Mask out everything outside the person
spill_mask *= foreground_mask.astype(np.float32)

# Convert image to HSV
hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)

# Convert hue to float
h_float = h.astype(np.float32)

# Red suppression
dist_to_red = np.minimum(np.abs(h_float - 0), np.abs(h_float - 179))
red_suppression = np.clip(dist_to_red / 20.0, 0.0, 1.0)
spill_mask *= red_suppression

# Apply hue shift to green and boost saturation near edges
delta = (60 - h_float + 90) % 180 - 90
h_blended = h_float + spill_mask * delta * 0.4
h_new = np.clip(h_blended, 0 , 179).astype(np.uint8)

# Saturation boost
s_boost = (spill_mask * saturation_boost).astype(np.uint8)
s_new = np.clip(s + s_boost, 0, 255)

# Merge channels and convert back to BGR
hsv_spill = cv2.merge([h_new, s_new, v])
bgr_spill = cv2.cvtColor(hsv_spill, cv2.COLOR_HSV2BGR)

cv2.imwrite("person_with_spill9.jpg", bgr_spill)

